In [1]:
# Create data for 1DCNN
import pandas as pd
import numpy as np
import os
import glob
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd
import wfdb
import statsmodels.api as sm
import matplotlib.image as mpimg
from matplotlib.figure import Figure
from IPython.display import display, HTML
import csv
from collections import defaultdict
from sklearn.metrics import r2_score
import matplotlib.ticker as ticker
from RDFunctions import *

In [2]:
def addSegmentDataToFrame(i,dfOuranno,starting_time,label,RRFrame,SpO2Frame,LabelFrame):
    LOWESSWINDOW = 3
    OUTLIERSD = 2.5
    record_name = dfOuranno.iloc[i]['RecordNum'] 
    flname = './FinalRecords/'+str(record_name)+'n.hea'
    recname = './FinalRecords/'+str(record_name)+'n'       
    [samples,R,S,H,firstline] = getIndexOfRRSpO2HR(flname)
    rec =  wfdb.io.rdrecord(str(recname))
    xrr = rec.p_signal[:,R]
    xspo2 = rec.p_signal[:,S]

    xrr = xrr[int(starting_time*60):int(starting_time*60+24*60)]
    xspo2 = xspo2[int(starting_time*60):int(starting_time*60+24*60)]
    t = np.arange(int(starting_time*60),int(starting_time*60+24*60),1)
    
    if(len(xrr) < 24*60):
        xrr = np.concatenate((xrr,np.repeat(np.nan,(24*60-len(xrr)))))
        xspo2 = np.concatenate((xspo2,np.repeat(np.nan,(24*60-len(xspo2)))))

    [xrrnew,trrnew] = outlierRejector(xrr,t,OUTLIERSD, default=15.0)
    [xspo2new,tspo2new] = outlierRejector(xspo2,t,OUTLIERSD, default=98.0)

    zrrnew = applyLowess(xrrnew,trrnew,LOWESSWINDOW*60)
    zspo2new = applyLowess(xspo2new,tspo2new,LOWESSWINDOW*60)
    rr = zrrnew[:,1]
    spo2 = zspo2new[:,1]

    if ((not np.isnan(rr).any()) and (not np.isnan(spo2).any()) and  len(rr)==60*24 and len(spo2)==60*24):
        seg_num = np.shape(RRFrame)[0]
        last = np.shape(RRFrame)[1]
        seg_id = 'Seg'+str(seg_num)
        RRFrame.loc[seg_num,1:last] = rr
        RRFrame.loc[seg_num,'SegId'] = seg_id
        SpO2Frame.loc[seg_num,1:last] = spo2
        SpO2Frame.loc[seg_num,'SegId'] = seg_id
        LabelFrame.loc[seg_num,:] = int(label)

In [3]:
annofile = r'RecordsAnno4.csv'
dfOuranno = pd.read_csv(annofile, encoding='iso-8859-1')
# Create column names for RR and SpO2
d = ['SegId']
for i in range(60*24):
    fieldName = 'RR_'+str(i)
    d.append(fieldName)
RRFrame = pd.DataFrame(columns = d)

d =['SegId']
for i in range(60*24):
    fieldName = 'SP_'+str(i)
    d.append(fieldName)
SpO2Frame = pd.DataFrame(columns = d)
LabelFrame = pd.DataFrame(columns = ['Label'])

RECORDS = np.shape(dfOuranno)[0]

for i in range(RECORDS):
    if ((dfOuranno.iloc[i]['Category']=='A' or dfOuranno.iloc[i]['Category']=='E')): 
        print(i,end=' ')
        mystr = dfOuranno.iloc[i]['DTW_RDSegs'].split(';')
        mystr = list(filter(None, mystr))
        for s in mystr:
            start = float(s.split(':')[0])
            addSegmentDataToFrame(i,dfOuranno,start,1,RRFrame,SpO2Frame,LabelFrame)

for i in range(RECORDS):
    if ((dfOuranno.iloc[i]['Category']=='C' or dfOuranno.iloc[i]['Category']=='F')): 
        print(i,end=' ')
        mystr = dfOuranno.iloc[i]['DTW_RDSegs'].split(';')
        mystr = list(filter(None, mystr)) 
        for s in mystr:
            start = float(s.split(':')[0])
            addSegmentDataToFrame(i,dfOuranno,start,0,RRFrame,SpO2Frame,LabelFrame)

13 19 23 25 39 43 45 46 50 54 66 67 69 70 78 82 86 87 91 93 114 116 117 122 128 130 132 139 151 158 160 162 166 168 169 176 178 181 183 185 188 200 206 207 209 212 213 218 227 229 232 238 248 253 267 270 275 299 300 320 323 328 333 342 344 346 352 354 356 360 362 365 367 368 373 376 379 382 383 385 386 389 391 397 403 404 412 415 417 423 428 432 436 437 442 450 455 462 464 469 475 476 490 492 493 508 513 515 520 523 524 545 548 554 555 556 557 562 563 566 568 571 575 585 598 601 610 612 613 614 620 623 630 645 650 651 653 654 662 670 677 679 680 697 701 706 708 712 720 723 734 735 737 744 750 752 758 760 766 767 772 779 785 790 791 794 803 812 815 816 818 823 824 825 826 836 837 838 839 845 857 859 860 867 870 885 887 890 893 894 900 904 908 909 2 3 4 5 8 11 16 18 20 21 29 34 35 36 48 52 64 74 76 88 89 90 95 96 98 102 103 105 106 112 113 120 123 125 126 127 133 136 138 140 146 149 150 152 155 156 157 159 161 167 170 171 172 179 182 189 198 199 204 208 210 222 228 231 234 236 237 241 24

In [5]:
print(np.shape(RRFrame))
print(np.shape(SpO2Frame))
print(np.shape(LabelFrame))
print(sum(LabelFrame.Label))

(1510, 1441)
(1510, 1441)
(1510, 1)
752.0


In [9]:
# Just to check we had same number of samples of machine learning
df = pd.read_csv('SegmentFeatures2.csv')
print(np.shape(df))
print(sum(df.Type1Pattern))

(1510, 17)
752


In [11]:
RRFrame.head()

,SegId,RR_0,RR_1,RR_2,RR_3,RR_4,RR_5,RR_6,RR_7,RR_8,...,RR_1430,RR_1431,RR_1432,RR_1433,RR_1434,RR_1435,RR_1436,RR_1437,RR_1438,RR_1439
0,Seg0,18.2714,18.256,18.2406,18.2252,18.2097,18.1943,18.1789,18.1634,18.148,...,16.2911,16.2502,16.2094,16.1687,16.1282,16.0877,16.0474,16.0072,15.9672,15.9273
1,Seg1,16.0849,16.1058,16.1267,16.1477,16.1688,16.19,16.2113,16.2326,16.2541,...,19.0427,19.0588,19.0749,19.091,19.107,19.1229,19.1388,19.1547,19.1705,19.1863
2,Seg2,18.9139,18.9024,18.8908,18.879,18.867,18.8549,18.8426,18.8302,18.8176,...,17.8764,17.869,17.8616,17.8543,17.8469,17.8396,17.8324,17.8251,17.8179,17.8108
3,Seg3,18.5439,18.5468,18.5498,18.5527,18.5555,18.5583,18.5611,18.5639,18.5667,...,21.2806,21.3178,21.3551,21.3924,21.4297,21.4671,21.5045,21.5419,21.5794,21.6168
4,Seg4,14.068,14.0903,14.1124,14.1343,14.156,14.1775,14.1988,14.2199,14.2408,...,15.9269,15.8972,15.8674,15.8377,15.8079,15.7782,15.7484,15.7186,15.6889,15.6591


In [12]:
SpO2Frame.head()

,SegId,SP_0,SP_1,SP_2,SP_3,SP_4,SP_5,SP_6,SP_7,SP_8,...,SP_1430,SP_1431,SP_1432,SP_1433,SP_1434,SP_1435,SP_1436,SP_1437,SP_1438,SP_1439
0,Seg0,95.4921,95.4893,95.4865,95.4836,95.4806,95.4776,95.4745,95.4713,95.468,...,95.6861,95.6927,95.6995,95.7066,95.7138,95.7212,95.7288,95.7366,95.7446,95.7528
1,Seg1,95.441,95.4324,95.4239,95.4155,95.4071,95.3988,95.3906,95.3824,95.3743,...,96.42,96.4282,96.4363,96.4443,96.4523,96.4602,96.4681,96.4759,96.4836,96.4912
2,Seg2,95.3987,95.3845,95.3703,95.3561,95.3419,95.3276,95.3134,95.2991,95.2849,...,95.5902,95.5819,95.5736,95.5653,95.557,95.5486,95.5402,95.5317,95.5232,95.5147
3,Seg3,95.0062,95.028,95.0499,95.0717,95.0935,95.1152,95.1368,95.1585,95.18,...,94.8972,94.9119,94.9265,94.941,94.9555,94.9698,94.984,94.9982,95.0122,95.0262
4,Seg4,98.7218,98.7132,98.7046,98.696,98.6875,98.679,98.6704,98.6619,98.6534,...,94.6177,94.6051,94.5923,94.5795,94.5666,94.5537,94.5406,94.5275,94.5143,94.5011


In [13]:
LabelFrame.head()

,Label
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [14]:
RRFrame.to_csv('RRFrame.csv',sep=',',index=False)
SpO2Frame.to_csv('SpO2Frame.csv',sep=',',index=False)
LabelFrame.to_csv('LabelFrame.csv',sep=',',index=False)